In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Beautiful format for float type
pd.set_option('display.float_format', lambda x: '%.3f' % x)

credit_card_txn = pd.read_csv('tj_05_credit_card_transaction.csv')
data_training = pd.read_csv('tj_05_training.csv', header=None, names=["card_no", "gender"])
data_test = pd.read_csv('tj_05_test.csv', header=None, names=["card_no"])

In [38]:
print('credit_card_txn shape =', credit_card_txn.shape, '\n')
credit_card_txn.info()
credit_card_txn.describe()

credit_card_txn shape = (893761, 6) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893761 entries, 0 to 893760
Data columns (total 6 columns):
card_no         893761 non-null int64
txn_date        884013 non-null object
txn_hour        884013 non-null float64
txn_amount      893761 non-null float64
mer_cat_code    884013 non-null float64
mer_id          893761 non-null int64
dtypes: float64(3), int64(2), object(1)
memory usage: 40.9+ MB


,card_no,txn_hour,txn_amount,mer_cat_code,mer_id
count,893761.000,884013.000,893761.000,884013.000,893761.000
mean,1234000000030626.250,14.466,1694.535,5803.844,7269.036
std,18378.600,4.670,25873.996,877.350,10765.232
min,1234000000000001.000,0.000,50.000,742.000,0.000
25%,1234000000007268.000,11.000,250.000,5411.000,0.000
50%,1234000000014186.000,15.000,450.000,5631.000,0.000
75%,1234000000021106.000,18.000,1000.000,6011.000,14070.000
max,1234000000028172.000,23.000,19201000.000,9405.000,28728.000


In [39]:
credit_card_txn = credit_card_txn.dropna(axis=0, how='any')
print("Dropped na shape =", credit_card_txn.shape, '\n')

credit_card_txn.sample(5)

Dropped na shape = (884013, 6) 



,card_no,txn_date,txn_hour,txn_amount,mer_cat_code,mer_id
695234,1234000000021149,2016-02-11 00:00:00,6.000,200.000,4812.000,16783
205598,1234000000007688,2016-04-12 00:00:00,16.000,400.000,5541.000,0
94864,1234000000002269,2016-05-06 00:00:00,23.000,50.000,5735.000,0
270078,1234000000001319,2016-07-09 00:00:00,15.000,50.000,5735.000,0
250851,1234000000025961,2016-04-28 00:00:00,18.000,200.000,5541.000,0


In [40]:
mcc_codes = pd.read_csv('mcc_codes.csv')
credit_card_txn = credit_card_txn.join(mcc_codes[['mcc', 'irs_description', 'type_id']].set_index('mcc'), on='mer_cat_code')
credit_card_txn = credit_card_txn.dropna(axis=0, how='any')
credit_card_txn.type_id = credit_card_txn.type_id.astype(int)
credit_card_txn.head()

,card_no,txn_date,txn_hour,txn_amount,mer_cat_code,mer_id,irs_description,type_id
0,1234000000009154,2016-12-07 00:00:00,22.000,1550.000,4511.000,0,"Airlines, Air Carriers",30
1,1234000000017165,2016-12-07 00:00:00,22.000,250.000,6011.000,0,Automated Cash Disburse,170
2,1234000000000768,2016-12-07 00:00:00,22.000,250.000,8398.000,0,Charitable and Social Service Organizations - ...,268
3,1234000000018716,2016-12-07 00:00:00,22.000,50.000,5735.000,0,Record Stores,120
4,1234000000016652,2016-12-07 00:00:00,22.000,50.000,5735.000,0,Record Stores,120


In [41]:
# credit_card_txn.sample(500)

In [42]:
data_features = credit_card_txn
data_features = data_features.drop(['txn_date', 'txn_hour', 'mer_cat_code', 'mer_id', 'irs_description'], axis=1)
data_features.head()

,card_no,txn_amount,type_id
0,1234000000009154,1550.000,30
1,1234000000017165,250.000,170
2,1234000000000768,250.000,268
3,1234000000018716,50.000,120
4,1234000000016652,50.000,120


In [43]:
data_training.shape

data_training_cleaned = pd.read_csv('tj_05_training_cleaned.csv')
data_training_cleaned.head()

,card_no,gender,0,1
0,1234000000000000,1,3.000,4.000
1,1234000000000010,1,5.000,3.000
2,1234000000000020,1,3.000,2.000
3,1234000000000030,1,5.000,3.000
4,1234000000000040,1,4.000,4.000


In [44]:
data_for_model = data_training.join(data_features.set_index('card_no'), on='card_no')
data_for_model = data_for_model.dropna(axis=0, how='any')
data_for_model.head()

,card_no,gender,txn_amount,type_id
0,1234000000000792,1,600.000,91.000
0,1234000000000792,1,7750.000,257.000
0,1234000000000792,1,2000.000,36.000
0,1234000000000792,1,500.000,91.000
0,1234000000000792,1,500.000,91.000


In [61]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.cross_validation import  train_test_split
from matplotlib import pyplot
from sklearn import metrics
from sklearn.cross_validation import KFold, cross_val_score
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
from time import time
from sklearn.preprocessing import StandardScaler

dataset = data_for_model
dataset = dataset.drop("card_no",1)
# dataset.head()
features = list(dataset.columns[1:])
print(features)

['txn_amount', 'type_id']


In [59]:
XGBmodel = XGBClassifier()
X_train, X_test, y_train, y_test = train_test_split(dataset[features], dataset['gender'].values, test_size=0.30, random_state=42)
XGBmodel.fit(X_train,y_train)
predictions = XGBmodel.predict(X_test)     

print ("accuracy" , metrics.accuracy_score(y_test, predictions))

accuracy 0.567601144407


In [76]:
model = Sequential()
# model.add(Dense(input_shape=2, activation='relu'))
model.add(Dense(12, input_dim=2, kernel_initializer ='uniform', activation='relu'))
model.add(Activation("softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [77]:
scaler = StandardScaler()
model.fit(scaler.fit_transform(X_train.values), y_train)

Epoch 1/10
514618/514618 [==============================] - 46s - loss: 0.7866 - acc: 0.5259    
Epoch 2/10
514618/514618 [==============================] - 51s - loss: 0.7010 - acc: 0.5263    
Epoch 3/10
514618/514618 [==============================] - 45s - loss: 0.6971 - acc: 0.5263    
Epoch 4/10
514618/514618 [==============================] - 49s - loss: 0.6956 - acc: 0.5263    
Epoch 5/10
514618/514618 [==============================] - 44s - loss: 0.6948 - acc: 0.5262    
Epoch 6/10
514618/514618 [==============================] - 49s - loss: 0.6942 - acc: 0.5263    
Epoch 7/10
514618/514618 [==============================] - 50s - loss: 0.6939 - acc: 0.5261    
Epoch 8/10
514618/514618 [==============================] - 38s - loss: 0.6936 - acc: 0.5261    
Epoch 9/10
514618/514618 [==============================] - 41s - loss: 0.6934 - acc: 0.5262    - ET
Epoch 10/10
514618/514618 [==============================] - 54s - loss: 0.6932 - acc: 0.5263    


In [78]:
y_prediction = model.predict_classes(scaler.transform(X_test.values))
print ("\n\naccuracy" , np.sum(y_prediction == y_test) / float(len(y_test)))

219744/220551 [============================>.] - ETA: 0s

accuracy 0.527243131974


In [107]:
model = Sequential()
model.add(Dense(3, input_dim=2))
# model.add(Dense(3, input_dim=2))
# model.add(Dense(3, input_dim=2))
model.add(Activation("softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model.fit(scaler.fit_transform(X_train.values), y_train)

Epoch 1/10
514618/514618 [==============================] - 59s - loss: 0.7132 - acc: 0.5248    
Epoch 2/10
157824/514618 [========>.....................] - ETA: 39s - loss: 0.6957 - acc: 0.5237

In [87]:
data_test.join()

TypeError: join() missing 1 required positional argument: 'other'

In [82]:

data_test_prediction = model.predict_classes(scaler.transform(data_test.values))

C:\Users\Watchanan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


ValueError: non-broadcastable output operand with shape (4623,1) doesn't match the broadcast shape (4623,2)